# Monarch Watch Data all taggings GBIF

In [3]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-11-2022

In [4]:
MWatch_Tagged_df = pd.read_csv("MWatch_occurrences_1996_2000.csv")
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLatitude,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus


In [5]:
MWatch_Tagged_df.columns.tolist()

['id',
 'modified',
 'language',
 'license',
 'institutionID',
 'collectionID',
 'institutionCode',
 'collectionCode',
 'datasetName',
 'basisOfRecord',
 'occurrenceID',
 'catalogNumber',
 'recordedBy',
 'occurrenceStatus',
 'associatedReferences',
 'eventDate',
 'year',
 'country',
 'stateProvince',
 'verbatimLocality',
 'decimalLatitude',
 'decimalLongitude',
 'geodeticDatum',
 'georeferenceRemarks',
 'taxonID',
 'nameAccordingToID',
 'scientificName',
 'kingdom',
 'vernacularName',
 'taxonRemarks']

In [6]:
def df_to_geojson(df, properties, lat='decimalLatitude', lon='decimalLongitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [7]:
MWatch_Tagged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363582 entries, 0 to 363581
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    363582 non-null  object 
 1   modified              363582 non-null  object 
 2   language              363582 non-null  object 
 3   license               363582 non-null  object 
 4   institutionID         363582 non-null  object 
 5   collectionID          363582 non-null  object 
 6   institutionCode       363582 non-null  object 
 7   collectionCode        363582 non-null  object 
 8   datasetName           363582 non-null  object 
 9   basisOfRecord         363582 non-null  object 
 10  occurrenceID          363582 non-null  object 
 11  catalogNumber         363582 non-null  object 
 12  recordedBy            363582 non-null  object 
 13  occurrenceStatus      363582 non-null  object 
 14  associatedReferences  363582 non-null  object 
 15  

In [8]:
MWatch_Tagged_df.describe()

,year,decimalLatitude,decimalLongitude,taxonID
count,363582.000000,363582.000000,363582.000000,363582.0
mean,1997.858071,39.694501,-90.112869,117273.0
std,1.763730,3.769477,8.696296,0.0
min,1992.000000,25.721490,-118.997683,117273.0
25%,1997.000000,38.203070,-96.731103,117273.0
50%,1998.000000,39.216660,-93.664964,117273.0
75%,1999.000000,42.213930,-82.491268,117273.0
max,2000.000000,49.895140,-67.958850,117273.0


In [9]:
print(MWatch_Tagged_df.shape)

(363582, 30)


In [10]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].unique())

['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['7/30/2019']
['English']
['https://creativecommons.org/publicdomain/zero/1.0/legalcode']
['https://bison.usgs.gov/']
['https://bison.usgs.gov/ipt/resource?r=monarchwatch']
['BISON']
['Monarch Watch']
['Monarch Watch']
['humanobservation']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['Monarch Watch']
['present']
['Monarch Watch Tagging Data, https://monarchwatch.org/tagdata']
['8/9/1992' '8/10/1992' '9/1/1992' ... '12/29/2000' '12/30/2000'
 '12/31/2000']
[1992 1993 1994 1995 1996 1997 1998 1999 2000]
['US' 'CA']
['MD' 'MO' 'KS' 'MI' 'KY' 'CT' 'NJ' 'CO' 'TX' 'OH' 'NY' 'MA' 'GA' 'IN'
 'PA' 'NC' 'MN' 'ON' 'NE' 'OK' 'FL' 'NH' 'VT' 'IA' 'IL' 'QC' 'WI' 'AZ'
 'VA' 'ME' 'WV' 'TN' 'SD' 'LA' 'RI' 'DE' 'AL' 'NM' 'AR' 'SC' 'ND' 'DC'
 'MS' 'MB' 'WY' 'BC' 'UT' 'ID']
['City:PLUM POINT' 'City:INDEPENDENCE' 'City

In [11]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].value_counts())

MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: id, Length: 363582, dtype: int64
7/30/2019    363582
Name: modified, dtype: int64
English    363582
Name: language, dtype: int64
https://creativecommons.org/publicdomain/zero/1.0/legalcode    363582
Name: license, dtype: int64
https://bison.usgs.gov/    363582
Name: institutionID, dtype: int64
https://bison.usgs.gov/ipt/resource?r=monarchwatch    363582
Name: collectionID, dtype: int64
BISON    363582
Name: institutionCode, dtype: int64
Monarch Watch    363582
Name: collectionCode, dtype: int64
Monarch Watch    363582
Name: datasetName, dtype: int64
humanobservation    363582
Name: basisOfRecord, dtype: int64
MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: occurrenceID, Length: 363582, dtype: int6

In [12]:
# Find null values
for column in MWatch_Tagged_df.columns:
   print(f"Column {column} has {MWatch_Tagged_df[column].isnull().sum()} null values")

Column id has 0 null values
Column modified has 0 null values
Column language has 0 null values
Column license has 0 null values
Column institutionID has 0 null values
Column collectionID has 0 null values
Column institutionCode has 0 null values
Column collectionCode has 0 null values
Column datasetName has 0 null values
Column basisOfRecord has 0 null values
Column occurrenceID has 0 null values
Column catalogNumber has 0 null values
Column recordedBy has 0 null values
Column occurrenceStatus has 0 null values
Column associatedReferences has 0 null values
Column eventDate has 0 null values
Column year has 0 null values
Column country has 0 null values
Column stateProvince has 0 null values
Column verbatimLocality has 0 null values
Column decimalLatitude has 0 null values
Column decimalLongitude has 0 null values
Column geodeticDatum has 0 null values
Column georeferenceRemarks has 0 null values
Column taxonID has 0 null values
Column nameAccordingToID has 0 null values
Column scienti

In [13]:
# Find duplicate entries
print(f"Duplicate entries: {MWatch_Tagged_df.duplicated().sum()}")

Duplicate entries: 0


# Format file for geoJSON

In [14]:
MWatch_Tagged_df['species'] = 'Monarch Tagged'

In [15]:
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks,species
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged


In [16]:
MWatch_Tagged_df.drop(['modified', 'language', 'license', 'institutionID', 'collectionID', 'institutionCode', 'collectionCode', 'datasetName', 'basisOfRecord', 'occurrenceID', 'catalogNumber', 'recordedBy', 'occurrenceStatus', 'associatedReferences', 'verbatimLocality', 'geodeticDatum', 'georeferenceRemarks', 'taxonID', 'nameAccordingToID','scientificName','kingdom', 'vernacularName','taxonRemarks'], axis=1, inplace=True)
MWatch_Tagged_df.head()

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged


In [17]:
cols = ['eventDate', 'species','country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'number']

In [18]:
MWatch_Tagged_df.shape

(363582, 8)

In [19]:
# MWatch_Tagged_df.replace(np.nan,'',regex=True)

In [20]:
MWatch_Tagged_df.shape

(363582, 8)

In [21]:
MWatch_Tagged_df

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
...,...,...,...,...,...,...,...,...
363577,MW0363578,12/30/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363578,MW0363579,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363579,MW0363580,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363580,MW0363581,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged


In [22]:
# MWatch_Tagged_df['column name'] = df['column name'].replace(['old value'],'new value')

In [23]:
MWatch_Tagged_df =MWatch_Tagged_df.groupby(['eventDate', 'year', 'country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'species']).size().reset_index()
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,0
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [24]:
MWatch_Tagged_df.rename( columns={0:'number'} ,inplace=True)
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [25]:
MWatch_Tagged_year_df = MWatch_Tagged_df
# MWatch_Tagged_year_df['years'] = pd.DatetimeIndex(MWatch_Tagged_year_df['eventDate']).year

# Filter Tagging Data By Year - 2000

In [26]:
# filtering by year
MWatch_Tagged_2000_df = MWatch_Tagged_df[MWatch_Tagged_df['year']==2000]
MWatch_Tagged_2000_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
5,1/10/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,10
6,1/10/2000,2000,US,TX,28.80527,-97.003598,Monarch Tagged,1
9,1/11/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,5


In [27]:
MWatch_Tagged_2000_df['eventDate']

0         1/1/2000
1         1/1/2000
5        1/10/2000
6        1/10/2000
9        1/11/2000
           ...    
40063     9/9/2000
40064     9/9/2000
40065     9/9/2000
40066     9/9/2000
40067     9/9/2000
Name: eventDate, Length: 9169, dtype: object

In [28]:
MWatch_Tagged_2000_df.shape

(9169, 8)

In [29]:
MWatch_Tagged_2000_df = pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_df['month'] = pd.DatetimeIndex(MWatch_Tagged_2000_df['eventDate']).month

In [30]:
MWatch_Tagged_2000_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1,1
5,1/10/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,10,1
6,1/10/2000,2000,US,TX,28.80527,-97.003598,Monarch Tagged,1,1
9,1/11/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,5,1


In [31]:
MWatch_Tagged_2000_df.drop(['year'], axis=1, inplace=True)

In [32]:
MWatch_Tagged_2000_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
0,1/1/2000,US,FL,28.01863,-82.112864,Monarch Tagged,1,1
1,1/1/2000,US,FL,28.53834,-81.379237,Monarch Tagged,1,1
5,1/10/2000,US,FL,28.53834,-81.379237,Monarch Tagged,10,1
6,1/10/2000,US,TX,28.80527,-97.003598,Monarch Tagged,1,1
9,1/11/2000,US,FL,28.53834,-81.379237,Monarch Tagged,5,1


## January

In [33]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_1_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==1]
MWatch_Tagged_2000_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
0,1/1/2000,US,FL,28.01863,-82.112864,Monarch Tagged,1,1
1,1/1/2000,US,FL,28.53834,-81.379237,Monarch Tagged,1,1
5,1/10/2000,US,FL,28.53834,-81.379237,Monarch Tagged,10,1
6,1/10/2000,US,TX,28.80527,-97.003598,Monarch Tagged,1,1
9,1/11/2000,US,FL,28.53834,-81.379237,Monarch Tagged,5,1


In [34]:
MWatch_Tagged_2000_1_df.shape

(60, 8)

In [35]:
MWatch_Tagged_2000_1_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
0,1/1/2000,US,FL,28.01863,-82.112864,Monarch Tagged,1,1
1,1/1/2000,US,FL,28.53834,-81.379237,Monarch Tagged,1,1
5,1/10/2000,US,FL,28.53834,-81.379237,Monarch Tagged,10,1
6,1/10/2000,US,TX,28.80527,-97.003598,Monarch Tagged,1,1
9,1/11/2000,US,FL,28.53834,-81.379237,Monarch Tagged,5,1
10,1/11/2000,US,TX,28.80527,-97.003598,Monarch Tagged,7,1
13,1/12/2000,US,FL,28.53834,-81.379237,Monarch Tagged,75,1
14,1/12/2000,US,FL,28.67000,-81.208120,Monarch Tagged,1,1
15,1/12/2000,US,SC,32.65518,-79.940367,Monarch Tagged,6,1
16,1/12/2000,US,SC,32.71183,-79.952113,Monarch Tagged,1,1


In [36]:
MWatch_Tagged_2000_1_df = pd.DataFrame(MWatch_Tagged_2000_1_df)
MWatch_Tagged_2000_1_df.drop(['month'], axis=1, inplace=True)

In [38]:
MWatch_Tagged_2000_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
0,1/1/2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
5,1/10/2000,US,FL,28.53834,-81.379237,Monarch Tagged,10
6,1/10/2000,US,TX,28.80527,-97.003598,Monarch Tagged,1
9,1/11/2000,US,FL,28.53834,-81.379237,Monarch Tagged,5


In [39]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_1_df['decimalLatitude'] = MWatch_Tagged_2000_1_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_1_df['decimalLongitude'] = MWatch_Tagged_2000_1_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [40]:
geojson = df_to_geojson(MWatch_Tagged_2000_1_df, cols)

In [41]:
file = open("MWatch_Tagged_2000_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [48]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_2_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==2]
MWatch_Tagged_2000_2_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11563,2/6/2000,US,TX,29.76043,-95.369803,Monarch Tagged,2,2
11564,2/7/2000,US,TX,29.56412,-95.025484,Monarch Tagged,1,2
11565,2/7/2000,US,TX,29.76043,-95.369803,Monarch Tagged,4,2
11567,2/9/2000,US,FL,28.53834,-81.379237,Monarch Tagged,1,2
11568,2/9/2000,US,TX,29.76043,-95.369803,Monarch Tagged,1,2


In [49]:
MWatch_Tagged_2000_2_df.shape

(28, 8)

In [50]:
MWatch_Tagged_2000_2_df = pd.DataFrame(MWatch_Tagged_2000_2_df)
MWatch_Tagged_2000_2_df.drop(['month'], axis=1, inplace=True)

In [51]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_2_df['decimalLatitude'] = MWatch_Tagged_2000_2_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_2_df['decimalLongitude'] = MWatch_Tagged_2000_2_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [52]:
geojson = df_to_geojson(MWatch_Tagged_2000_2_df, cols)

In [53]:
file = open("MWatch_Tagged_2000_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [54]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_3_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==3]
MWatch_Tagged_2000_3_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11637,3/8/2000,US,FL,28.38862,-81.565907,Monarch Tagged,6,3
11638,3/8/2000,US,FL,28.53834,-81.379237,Monarch Tagged,3,3
11640,3/9/2000,US,FL,28.01863,-82.112864,Monarch Tagged,2,3
11641,3/9/2000,US,FL,28.53834,-81.379237,Monarch Tagged,1,3
11642,3/9/2000,US,TX,29.50745,-95.094930,Monarch Tagged,3,3


In [55]:
MWatch_Tagged_2000_3_df.shape

(65, 8)

In [56]:
MWatch_Tagged_2000_3_df = pd.DataFrame(MWatch_Tagged_2000_3_df)
MWatch_Tagged_2000_3_df.drop(['month'], axis=1, inplace=True)

In [57]:
MWatch_Tagged_2000_3_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11569,3/1/2000,US,FL,28.371449,-81.557693,Monarch Tagged,2
11570,3/1/2000,US,FL,28.388620,-81.565907,Monarch Tagged,2
11571,3/1/2000,US,FL,28.670000,-81.208120,Monarch Tagged,2
11572,3/10/2000,US,FL,28.670000,-81.208120,Monarch Tagged,1
11573,3/10/2000,US,TX,29.507450,-95.094930,Monarch Tagged,8


In [58]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_3_df['decimalLatitude'] = MWatch_Tagged_2000_3_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_3_df['decimalLongitude'] = MWatch_Tagged_2000_3_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [59]:
geojson = df_to_geojson(MWatch_Tagged_2000_3_df, cols)

In [60]:
file = open("MWatch_Tagged_2000_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [61]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_4_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==4]
MWatch_Tagged_2000_4_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11729,4/4/2000,US,FL,28.53834,-81.379237,Monarch Tagged,2,4
11731,4/5/2000,US,FL,30.43826,-84.280733,Monarch Tagged,2,4
11732,4/5/2000,US,TX,29.50745,-95.094930,Monarch Tagged,2,4
11737,4/7/2000,US,TX,29.50745,-95.094930,Monarch Tagged,1,4
11739,4/8/2000,US,TX,29.50745,-95.094930,Monarch Tagged,4,4


In [62]:
MWatch_Tagged_2000_4_df.shape

(31, 8)

In [63]:
MWatch_Tagged_2000_4_df = pd.DataFrame(MWatch_Tagged_2000_4_df)
MWatch_Tagged_2000_4_df.drop(['month'], axis=1, inplace=True)

In [64]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_4_df['decimalLatitude'] = MWatch_Tagged_2000_4_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_4_df['decimalLongitude'] = MWatch_Tagged_2000_4_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [65]:
geojson = df_to_geojson(MWatch_Tagged_2000_4_df, cols)

In [66]:
file = open("MWatch_Tagged_2000_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [67]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_5_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==5]
MWatch_Tagged_2000_5_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11822,5/4/2000,US,KS,39.183608,-96.571669,Monarch Tagged,1,5
11837,5/7/2000,US,TX,29.507450,-95.094930,Monarch Tagged,5,5
11841,5/8/2000,US,FL,30.438260,-84.280733,Monarch Tagged,3,5
11845,5/9/2000,US,FL,28.018630,-82.112864,Monarch Tagged,1,5
11846,5/9/2000,US,FL,30.438260,-84.280733,Monarch Tagged,3,5


In [68]:
MWatch_Tagged_2000_5_df.shape

(30, 8)

In [69]:
MWatch_Tagged_2000_5_df =pd.DataFrame(MWatch_Tagged_2000_5_df)
MWatch_Tagged_2000_5_df.drop(['month'], axis=1, inplace=True)

In [70]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_5_df['decimalLatitude'] = MWatch_Tagged_2000_5_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_5_df['decimalLongitude'] = MWatch_Tagged_2000_5_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [71]:
geojson = df_to_geojson(MWatch_Tagged_2000_5_df, cols)

In [72]:
file = open("MWatch_Tagged_2000_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [73]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_6_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==6]
MWatch_Tagged_2000_6_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11881,6/24/2000,US,FL,28.01863,-82.112864,Monarch Tagged,21,6
11882,6/25/2000,US,FL,28.01863,-82.112864,Monarch Tagged,8,6
11884,6/26/2000,US,FL,28.01863,-82.112864,Monarch Tagged,1,6
11889,6/3/2000,US,PA,40.67491,-75.617899,Monarch Tagged,7,6
11892,6/4/2000,US,PA,40.67491,-75.617899,Monarch Tagged,6,6


In [74]:
MWatch_Tagged_2000_6_df.shape

(14, 8)

In [75]:
MWatch_Tagged_2000_6_df =pd.DataFrame(MWatch_Tagged_2000_6_df)
MWatch_Tagged_2000_6_df.drop(['month'], axis=1, inplace=True)

In [77]:
MWatch_Tagged_2000_6_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11849,6/1/2000,US,PA,40.67491,-75.617899,Monarch Tagged,9
11853,6/10/2000,US,CT,41.27055,-72.946971,Monarch Tagged,1
11854,6/11/2000,US,CT,41.27055,-72.946971,Monarch Tagged,1
11856,6/12/2000,US,CT,41.27055,-72.946971,Monarch Tagged,2
11863,6/14/2000,US,CT,41.27055,-72.946971,Monarch Tagged,1


In [78]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_6_df['decimalLatitude'] = MWatch_Tagged_2000_6_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_6_df['decimalLongitude'] = MWatch_Tagged_2000_6_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [79]:
geojson = df_to_geojson(MWatch_Tagged_2000_6_df, cols)

In [80]:
file = open("MWatch_Tagged_2000_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [81]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_7_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==7]
MWatch_Tagged_2000_7_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11918,7/22/2000,US,CT,41.27055,-72.946971,Monarch Tagged,4,7
11921,7/23/2000,US,CT,41.27055,-72.946971,Monarch Tagged,2,7
11925,7/26/2000,US,CT,41.27055,-72.946971,Monarch Tagged,2,7
11940,7/30/2000,CA,ON,43.77643,-79.231752,Monarch Tagged,1,7
11946,7/8/2000,US,KS,38.24640,-97.348923,Monarch Tagged,14,7


In [82]:
MWatch_Tagged_2000_7_df.shape

(10, 8)

In [84]:
MWatch_Tagged_2000_7_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11903,7/13/2000,US,CT,41.27055,-72.946971,Monarch Tagged,4,7
11904,7/13/2000,US,SD,43.54460,-96.731103,Monarch Tagged,2,7
11907,7/14/2000,US,CT,41.27055,-72.946971,Monarch Tagged,8,7
11909,7/16/2000,US,CT,41.27055,-72.946971,Monarch Tagged,3,7
11910,7/18/2000,US,CT,41.27055,-72.946971,Monarch Tagged,1,7


In [85]:
MWatch_Tagged_2000_7_df =pd.DataFrame(MWatch_Tagged_2000_7_df)
MWatch_Tagged_2000_7_df.drop(['month'], axis=1, inplace=True)

In [86]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_7_df['decimalLatitude'] = MWatch_Tagged_2000_7_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_7_df['decimalLongitude'] = MWatch_Tagged_2000_7_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [88]:
MWatch_Tagged_2000_7_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11903,7/13/2000,US,CT,41.27055,-72.946971,Monarch Tagged,4
11904,7/13/2000,US,SD,43.54460,-96.731103,Monarch Tagged,2
11907,7/14/2000,US,CT,41.27055,-72.946971,Monarch Tagged,8
11909,7/16/2000,US,CT,41.27055,-72.946971,Monarch Tagged,3
11910,7/18/2000,US,CT,41.27055,-72.946971,Monarch Tagged,1


In [89]:
geojson = df_to_geojson(MWatch_Tagged_2000_7_df, cols)

In [90]:
file = open("MWatch_Tagged_2000_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [91]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_8_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==8]
MWatch_Tagged_2000_8_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
16086,8/6/2000,US,MN,45.188850,-93.397454,Monarch Tagged,2,8
16087,8/6/2000,US,OK,35.222570,-97.439478,Monarch Tagged,1,8
16088,8/6/2000,US,SD,43.544600,-96.731103,Monarch Tagged,20,8
16119,8/8/2000,US,AR,35.868410,-92.117652,Monarch Tagged,32,8
16120,8/8/2000,US,IA,42.808036,-92.258234,Monarch Tagged,25,8


In [92]:
MWatch_Tagged_2000_8_df.shape

(1066, 8)

In [93]:
MWatch_Tagged_2000_8_df = pd.DataFrame(MWatch_Tagged_2000_8_df)
MWatch_Tagged_2000_8_df.drop(['month'], axis=1, inplace=True)

In [94]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_8_df['decimalLatitude'] = MWatch_Tagged_2000_8_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_8_df['decimalLongitude'] = MWatch_Tagged_2000_8_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [95]:
geojson = df_to_geojson(MWatch_Tagged_2000_8_df, cols)

In [96]:
file = open("MWatch_Tagged_2000_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [97]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_9_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==9]
MWatch_Tagged_2000_9_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
40063,9/9/2000,US,WI,44.02471,-88.542614,Monarch Tagged,9,9
40064,9/9/2000,US,WI,44.18582,-88.462609,Monarch Tagged,32,9
40065,9/9/2000,US,WI,44.23916,-87.641195,Monarch Tagged,2,9
40066,9/9/2000,US,WI,44.47749,-88.450657,Monarch Tagged,1,9
40067,9/9/2000,US,WY,44.68117,-106.992085,Monarch Tagged,1,9


In [98]:
MWatch_Tagged_2000_9_df.shape

(5380, 8)

In [99]:
MWatch_Tagged_2000_9_df = pd.DataFrame(MWatch_Tagged_2000_9_df)
MWatch_Tagged_2000_9_df.drop(['month'], axis=1, inplace=True)

In [100]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_9_df['decimalLatitude'] = MWatch_Tagged_2000_9_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_9_df['decimalLongitude'] = MWatch_Tagged_2000_9_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [101]:
geojson = df_to_geojson(MWatch_Tagged_2000_9_df, cols)

In [102]:
file = open("MWatch_Tagged_2000_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [103]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_10_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==10]
MWatch_Tagged_2000_10_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
10447,10/9/2000,US,WI,43.06057,-88.106479,Monarch Tagged,2,10
10448,10/9/2000,US,WI,43.07305,-89.401230,Monarch Tagged,1,10
10449,10/9/2000,US,WI,43.17418,-87.924254,Monarch Tagged,1,10
10450,10/9/2000,US,WI,43.91159,-88.728949,Monarch Tagged,1,10
10451,10/9/2000,US,WV,38.64289,-79.331153,Monarch Tagged,12,10


In [104]:
MWatch_Tagged_2000_10_df.shape

(2270, 8)

In [105]:
MWatch_Tagged_2000_10_df = pd.DataFrame(MWatch_Tagged_2000_10_df)
MWatch_Tagged_2000_10_df.drop(['month'], axis=1, inplace=True)

In [106]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_10_df['decimalLatitude'] = MWatch_Tagged_2000_10_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_10_df['decimalLongitude'] = MWatch_Tagged_2000_10_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [107]:
geojson = df_to_geojson(MWatch_Tagged_2000_10_df, cols)

In [108]:
file = open("MWatch_Tagged_2000_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [109]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_11_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==11]
MWatch_Tagged_2000_11_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11310,11/9/2000,US,FL,29.02832,-81.303118,Monarch Tagged,5,11
11311,11/9/2000,US,LA,29.95107,-90.071532,Monarch Tagged,4,11
11312,11/9/2000,US,SC,32.65518,-79.940367,Monarch Tagged,1,11
11313,11/9/2000,US,SC,32.71183,-79.952113,Monarch Tagged,11,11
11314,11/9/2000,US,TX,29.76043,-95.369803,Monarch Tagged,4,11


In [110]:
MWatch_Tagged_2000_11_df.shape

(182, 8)

In [111]:
MWatch_Tagged_2000_11_df = pd.DataFrame(MWatch_Tagged_2000_11_df)
MWatch_Tagged_2000_11_df.drop(['month'], axis=1, inplace=True)

In [112]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_11_df['decimalLatitude'] = MWatch_Tagged_2000_11_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_11_df['decimalLongitude'] = MWatch_Tagged_2000_11_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [113]:
geojson = df_to_geojson(MWatch_Tagged_2000_11_df, cols)

In [114]:
file = open("MWatch_Tagged_2000_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [115]:
MWatch_Tagged_2000_month_df =pd.DataFrame(MWatch_Tagged_2000_df)
MWatch_Tagged_2000_12_df = MWatch_Tagged_2000_month_df[MWatch_Tagged_2000_month_df['month']==12]
MWatch_Tagged_2000_12_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11506,12/7/2000,US,FL,28.39362,-81.538684,Monarch Tagged,4,12
11507,12/7/2000,US,TX,29.50745,-95.094930,Monarch Tagged,1,12
11508,12/7/2000,US,TX,29.76043,-95.369803,Monarch Tagged,1,12
11518,12/8/2000,US,TX,29.50745,-95.094930,Monarch Tagged,2,12
11531,12/9/2000,US,TX,29.50745,-95.094930,Monarch Tagged,1,12


In [116]:
MWatch_Tagged_2000_12_df.shape

(33, 8)

In [117]:
MWatch_Tagged_2000_12_df = pd.DataFrame(MWatch_Tagged_2000_12_df)
MWatch_Tagged_2000_12_df.drop(['month'], axis=1, inplace=True)

In [118]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_2000_12_df['decimalLatitude'] = MWatch_Tagged_2000_12_df['decimalLatitude'].astype(float)
MWatch_Tagged_2000_12_df['decimalLongitude'] = MWatch_Tagged_2000_12_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [119]:
geojson = df_to_geojson(MWatch_Tagged_2000_12_df, cols)

In [120]:
file = open("MWatch_Tagged_2000_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()